In [ ]:
import tensorflow as tf
import numpy as np
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.log_device_placement = True

# make & read data

In [5]:
dataset = tf.data.Dataset.from_tensor_slices([i for i in range(4)]).shuffle(buffer_size=8).repeat(2)
iter = dataset.make_one_shot_iterator()

with tf.Session() as sess:
    try:
        while True:
            print sess.run(iter.get_next())
    except:
        print "end"

<class 'tensorflow.python.data.ops.dataset_ops.RepeatDataset'>


### dataset Transformation : map shuffle batch

In [ ]:
dataset1 = dataset.map(lambda x : x + 1).batch(32).shuffle(buffer_size=100)
iter1 = dataset1.make_one_shot_iterator()
with tf.Session() as sess:
    try:
        while True:
            print sess.run(iter1.get_next())
    except:
        print "end"

# read TFRecord on disk

In [21]:
%%time
record_dataset = tf.data.TFRecordDataset(['/data01/user/caobaiyu/tensorflow/krRec/training.tfrecords'], buffer_size=100)
#record_dataset
it = record_dataset.make_one_shot_iterator()
rec = it.get_next()
features = tf.parse_single_example(rec,
            features = {
            'examp1': tf.FixedLenFeature([400], tf.float32),
            'examp2': tf.FixedLenFeature([400], tf.float32)
            })
sample1 = features['examp1']
sample2 = features['examp2']
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    num = 0
    try:
        while True:
            a,b = sess.run([sample1, sample2])
            num += 1
    except:
        print 'end, %d records in total.' % num

end, 541790 records in total.
CPU times: user 7min 3s, sys: 1min 8s, total: 8min 11s
Wall time: 4min 21s


In [24]:
%%time
record_dataset = tf.data.TFRecordDataset(['/data01/user/caobaiyu/tensorflow/krRec/training.tfrecords'], buffer_size=1000).batch(200)
#record_dataset
it = record_dataset.make_one_shot_iterator()
rec = it.get_next()
features = tf.parse_example(rec,
            features = {
            'examp1': tf.FixedLenFeature([400], tf.float32),
            'examp2': tf.FixedLenFeature([400], tf.float32)
            })
sample1 = features['examp1']
sample2 = features['examp2']
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    num = 0
    try:
        while num < 600000:
            a,b = sess.run([sample1, sample2])
            num += 200
    except:
        pass
    print 'end, %d records in total.' % num

end, 541800 records in total.
CPU times: user 6.35 s, sys: 1.45 s, total: 7.81 s
Wall time: 3.7 s
